In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
with open('/content/drive/My Drive/fr-en.en', "r") as f:
    english_fr_en = f.read()      

In [0]:
with open('/content/drive/My Drive/de-en.en', "r") as f:
    english_de_en = f.read()  
    

In [0]:
with open('/content/drive/My Drive/de-en.de', "r") as f:
    german_de_en = f.read()  

In [0]:
with open('/content/drive/My Drive/fr-en.fr', "r") as f:
    french_fr_en = f.read()  

In [0]:
from string import punctuation

In [0]:
import string
import nltk
import copy
english_fr_en_sentences = []
french_fr_en_sentences = []
english_de_en_sentences = []
german_de_en_sentences = []


# text pre processing
def Text_Preprocess(text, language):
    lower_case_text = text.lower()
    for sentence in lower_case_text.split('\n'):
        out = "".join(l for l in sentence if l not in string.punctuation)
        if language == 'english_fr_en':
            english_fr_en_sentences.append(out)
        elif language == 'french_fr_en':
            french_fr_en_sentences.append(out)
        elif language == 'english_de_en':
            english_de_en_sentences.append(out)
        elif language == 'german_de_en':
            german_de_en_sentences.append(out)

Text_Preprocess(english_fr_en, 'english_fr_en')
Text_Preprocess(french_fr_en, 'french_fr_en')
Text_Preprocess(english_de_en, 'english_de_en')
Text_Preprocess(german_de_en, 'german_de_en')

temp = []
for sentence in english_fr_en_sentences:
    words = sentence.split()
    for word in words:
        temp.append(word)
        
unique_english_fr_en_vocab = set(temp) # unique vocab english

temp = []
for sentence in french_fr_en_sentences:
    words = sentence.split()
    for word in words:
        temp.append(word)
        
unique_french_fr_en_vocab = set(temp) # unique vocab french

temp = []
for sentence in english_de_en_sentences:
    words = sentence.split()
    for word in words:
        temp.append(word)
        
unique_english_de_en_vocab = set(temp) # unique vocab english

temp = []
for sentence in german_de_en_sentences:
    words = sentence.split()
    for word in words:
        temp.append(word)
        
unique_german_de_en_vocab = set(temp) # unique german vocab


In [0]:
english_fr_Translation_probabiliy = {}
french_fr_Translation_probabiliy = {}

for french_word in unique_french_fr_en_vocab:
     french_fr_Translation_probabiliy[french_word] = (1/len(unique_english_fr_en_vocab))

for eng_word in unique_english_fr_en_vocab:
    english_fr_Translation_probabiliy[eng_word] = copy.deepcopy(french_fr_Translation_probabiliy)

In [0]:
english_de_Translation_probabiliy = {}
german_de_Translation_probabiliy = {}

for german_word in unique_german_de_en_vocab:
     german_de_Translation_probabiliy[german_word] = (1/len(unique_english_de_en_vocab))

for eng_word in unique_english_de_en_vocab:
    english_de_Translation_probabiliy[eng_word] = copy.deepcopy(german_de_Translation_probabiliy)

In [0]:
import copy
def init_parameters(foreign_language):
    if foreign_language == 'german':
        english_vocab = unique_english_de_en_vocab
        foreign_language_vocab = unique_german_de_en_vocab
        probability_english = english_de_Translation_probabiliy
        english_sentences = english_de_en_sentences
        foreign_sentences = german_de_en_sentences
    if foreign_language == 'french':
        english_vocab = unique_english_fr_en_vocab
        foreign_language_vocab = unique_french_fr_en_vocab
        probability_english = english_fr_Translation_probabiliy
        english_sentences = english_fr_en_sentences
        foreign_sentences = french_fr_en_sentences
        
    clone_translation_probability = copy.deepcopy(probability_english)

    return [english_vocab, foreign_language_vocab, probability_english,  english_sentences,
            foreign_sentences, 0.002, 100, clone_translation_probability]

def init_total_f(foreign_language_vocab):
    total_f = {}
    for each_vocab in foreign_language_vocab:
        total_f[each_vocab] = 0
    return total_f

def counts_vocab_english(english_vocab, total_f):
    count = {}
    for each_english_vocab in english_vocab:
        count[each_english_vocab] = copy.deepcopy(total_f)
    return count

def construct_sum_array(sentence_words_e, sentence_words_foreign, clone_translation_probability):
    sum_words = {}
    for english_word in sentence_words_e:
            sum_words[english_word] = 0
            for foreign_word in sentence_words_foreign:
                    sum_words[english_word] += clone_translation_probability[english_word][foreign_word]
    return sum_words

def construct_total_f_array(sentence_words_e, sentence_words_foreign, clone_translation_probability,  count, total_of_f, sum_words):
    for english_word in sentence_words_e:
        for foriegn_word in sentence_words_foreign:
                count[english_word][foriegn_word] += clone_translation_probability[english_word][foriegn_word]/sum_words[english_word]                                       
                total_of_f[foriegn_word] += clone_translation_probability[english_word][foriegn_word]/sum_words[english_word]

def update_translation_probabilites(foreign_language_vocab, english_vocab, clone_translation_probability, count, total_of_f):
    for foreign_word in foreign_language_vocab:
         for english_word in english_vocab:
                 clone_translation_probability[english_word][foreign_word] = count[english_word][foreign_word]/total_of_f[foreign_word]

def sum_of_differenceProbabilites(english_vocab, foreign_language_vocab, clone_translation_probability, precounter_probabilities):
        sum_prob = 0
        for(english_word, foreign_word) in zip(english_vocab, foreign_language_vocab):
                    difference_in_prob = (clone_translation_probability[english_word][foreign_word] - precounter_probabilities[english_word][foreign_word])**2
                    sum_prob += difference_in_prob
        sum_prob = sum_prob**0.5   
        return sum_prob

def calculate_word_translation_probabilites(foreign_language):
    english_vocab = None
    foreign_language_vocab = None
    probability_english = None
    english_senteces = None
    foreign_sentences = None
    
    result = init_parameters(foreign_language)
    english_vocab = result[0]
    foreign_language_vocab = result[1]
    probability_english = result[2]
    english_sentences = result[3]
    foreign_sentences = result[4]
    threshold_limit_convergence = result[5]
    sum_prob = result[6]
    total_of_f = {}
    iterations = 0
    clone_translation_probability = result[7]
    iterations = 0
    
    while(sum_prob> threshold_limit_convergence):
            precounter_probabilities = copy.deepcopy(clone_translation_probability)
            total_of_f = init_total_f(foreign_language_vocab)
            
            counts = counts_vocab_english(english_vocab, total_of_f)
                
            for e_sentence, foreign_sentence in zip(english_sentences, foreign_sentences):
                sentence_words_e = e_sentence.split()
                sentence_words_foreign = foreign_sentence.split()
                sum_words = construct_sum_array(sentence_words_e, sentence_words_foreign, clone_translation_probability)
                construct_total_f_array(sentence_words_e, sentence_words_foreign, clone_translation_probability,  counts, total_of_f, sum_words)
            update_translation_probabilites(foreign_language_vocab, english_vocab, clone_translation_probability, counts, total_of_f)
            
    
            sum_prob = sum_of_differenceProbabilites(english_vocab, foreign_language_vocab, clone_translation_probability, precounter_probabilities)
            iterations = iterations + 1
            print("Iteration - {} - convergence_value - {}".format(iterations , sum_prob) )
    return clone_translation_probability

       


In [11]:

french_translation_probabilities = calculate_word_translation_probabilites('french')

Iteration - 1 - convergence_value - 0.17954948386436964
Iteration - 2 - convergence_value - 0.07673196625776457
Iteration - 3 - convergence_value - 0.05548935524249861
Iteration - 4 - convergence_value - 0.039334085995729656
Iteration - 5 - convergence_value - 0.02761719741491891
Iteration - 6 - convergence_value - 0.024013695260283567
Iteration - 7 - convergence_value - 0.021689385725606067
Iteration - 8 - convergence_value - 0.01920970897936244
Iteration - 9 - convergence_value - 0.016878102249912093
Iteration - 10 - convergence_value - 0.01490543584709952
Iteration - 11 - convergence_value - 0.013263121601625365
Iteration - 12 - convergence_value - 0.01188277828819381
Iteration - 13 - convergence_value - 0.010716577997674491
Iteration - 14 - convergence_value - 0.009727772141728624
Iteration - 15 - convergence_value - 0.008883128095697211
Iteration - 16 - convergence_value - 0.0081535965741923
Iteration - 17 - convergence_value - 0.0075149678661236745
Iteration - 18 - convergence_va

In [12]:
for ew,french_data in french_translation_probabilities.items():
  for fw,pb in french_data.items():   
    if pb > 0.2:
      print("{}           {}           {}".format(ew, fw, pb))

address           devra           0.2645699798196816
address           précisément           0.4912664792687484
outermost           thématique           0.40928514892104284
treaty           traité           0.9907328094232852
he           leçon           0.22835064840885064
he           intervienne           0.23461657787740903
he           passée           0.20597778012543796
he           nette           0.22918592718578637
building           bâtiment           0.8080513142706213
seem           réalisable           0.3203304610155264
up           soulèverai           0.22503113051047222
up           cite           0.3129382849461101
up           élaborer           0.3443054486992585
up           veiller           0.22263996713363945
up           perspectives           0.3444751606342209
up           repris           0.5021997673314916
up           voulons           0.20508693541784406
up           instituées           0.24969422973256716
up           plafonnée           0.248225118111

In [0]:
def print_french_alignment():
  for sentence in french_fr_en_sentences:
    words = sentence.split()
    result_words = []
    
    for word in words:
      max_multiply = -70
      for trans_word in french_translation_probabilities.keys():
        if french_translation_probabilities[trans_word][word] > max_multiply:
            max_multiply = french_translation_probabilities[trans_word][word]
            max_word = trans_word
          
      result_words.append(max_word)
    print(result_words)
    print(sentence)


In [14]:
print_french_alignment()

['session', 'of', 'the', 'partsession']
reprise de la session
['i', 'pleasant', 'session', 'the', 'partsession', 'the', 'parliament', 'european', 'which', 'a', 'been', 'pleasant', 'the', 'friday', '17', 'december', 'like', 'and', 'i', 'you', 'pleasant', 'all', 'my', 'happy', 'in', 'pleasant', 'that', 'you', 'you', 'past', 'of', 'good', 'pleasant']
je déclare reprise la session du parlement européen qui avait été interrompue le vendredi 17 décembre dernier et je vous renouvelle tous mes vux en espérant que vous avez passé de bonnes vacances 
['as', 'you', 'you', 'been', 'the', 'that', 'the', 'in', 'materialise', 'of', 'the', 'were', 'failed', 'not', 'that', 'is', 'not', 'in', 'in', 'materialise', 'the', 'citizens', 'of', 'a', 'have', 'number', 'of', 'our', 'countries', 'have', 'been', 'number', 'of', 'disasters', 'natural', 'which', 'have', 'all', 'been', 'suffered']
comme vous avez pu le constater  le grand  bogue de l an 2000  ne s est pas produit  en revanche  les citoyens d un certa